In [2]:
# Run installation script
!source <(curl -fsSL https://raw.githubusercontent.com/jurgjn/alphafold3-polymer-bonds/refs/heads/main/colab/install)

Installing AlphaFold v3.0.1
Installing colabfold


In [3]:
# Copy weights from Google Drive to the default location at /root/models/af3.bin.zst
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /root/models/
!cp /content/drive/MyDrive/alphafold3/models/af3.bin.zst /root/models/af3.bin.zst
!md5sum /root/models/af3.bin.zst

Mounted at /content/drive
ea076e2d05168d66a3d9a3fa62e029d0  /root/models/af3.bin.zst


In [4]:
%%shell
mkdir -p /content/alphafold3_examples/fasta
cd /content/alphafold3_examples/fasta
wget https://rest.uniprot.org/uniprotkb/O00244.fasta
wget https://rest.uniprot.org/uniprotkb/P0CG48.fasta
wget https://rest.uniprot.org/uniprotkb/Q01973.fasta

--2025-09-19 08:07:43--  https://rest.uniprot.org/uniprotkb/O00244.fasta
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167 [text/plain]
Saving to: ‘O00244.fasta’

O00244.fasta        100%[===================>]     167  --.-KB/s    in 0s      

2025-09-19 08:07:44 (21.2 MB/s) - ‘O00244.fasta’ saved [167/167]

--2025-09-19 08:07:44--  https://rest.uniprot.org/uniprotkb/P0CG48.fasta
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775 [text/plain]
Saving to: ‘P0CG48.fasta’

P0CG48.fasta        100%[===================>]     775  --.-KB/s    in 0s      

2025-09-19 08:07:44 (727 MB/s) - ‘P0CG48.fasta’ saved [775/775]

--2025-09-19 08:07:44--  https://rest.uniprot.org/uniprotkb/Q01973.fast

In [5]:
%%shell
MPLBACKEND=AGG
source /colabfold_venv/bin/activate
mkdir -p /content/alphafold3_examples/colabfold_msas
colabfold_batch --msa-only --af3-json /content/alphafold3_examples/fasta/O00244.fasta /content/alphafold3_examples/colabfold_msas
colabfold_batch --msa-only --af3-json /content/alphafold3_examples/fasta/P0CG48.fasta /content/alphafold3_examples/colabfold_msas
colabfold_batch --msa-only --af3-json /content/alphafold3_examples/fasta/Q01973.fasta /content/alphafold3_examples/colabfold_msas

2025-09-19 08:07:48,578 Running colabfold 1.5.5 (e8ebd9a612b9da5fe12bbbc09513d5c52a9af633)

limited shared resource only capable of processing a few thousand MSAs per day. Please
submit jobs only from a single IP address. We reserve the right to limit access to the
server case-by-case when usage exceeds fair use. If you require more MSAs: You can 
precompute all MSAs with `colabfold_search` or host your own API and pass it to `--host-url`

COMPLETE: 100% 150/150 [00:01<00:00, 116.49it/s]
2025-09-19 08:07:51,415 Running colabfold 1.5.5 (e8ebd9a612b9da5fe12bbbc09513d5c52a9af633)

limited shared resource only capable of processing a few thousand MSAs per day. Please
submit jobs only from a single IP address. We reserve the right to limit access to the
server case-by-case when usage exceeds fair use. If you require more MSAs: You can 
precompute all MSAs with `colabfold_search` or host your own API and pass it to `--host-url`

COMPLETE: 100% 150/150 [00:01<00:00, 80.42it/s] 
2025-09-19 08:

In [6]:
!mkdir -p /content/alphafold3_examples/input
!jq -r '.name |= "O00244"' \
  /content/alphafold3_examples/colabfold_msas/sp_O00244_ATOX1_HUMAN_Copper_transport_protein_ATOX1_OS_Homo_sapiens_OX_9606_GN_ATOX1_PE_1_SV_1.json \
  > /content/alphafold3_examples/input/O00244.json
!jq -r '.name |= "P0CG48"' \
  /content/alphafold3_examples/colabfold_msas/sp_P0CG48_UBC_HUMAN_Polyubiquitin-C_OS_Homo_sapiens_OX_9606_GN_UBC_PE_1_SV_3.json \
  > /content/alphafold3_examples/input/P0CG48.json
!jq -r '.name |= "Q01973"' \
  /content/alphafold3_examples/colabfold_msas/sp_Q01973_ROR1_HUMAN_Inactive_tyrosine-protein_kinase_transmembrane_receptor_ROR1_OS_Homo_sapiens_OX_9606_GN_ROR1_PE_1_SV_2.json \
  > /content/alphafold3_examples/input/Q01973.json

In [7]:
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/alphafold3_examples/input/ \
  --output_dir=/content/alphafold3_examples/output \
  --norun_data_pipeline

Updated 0 paths from the index
Compilation Time Workaround with XLA Flags
GPU capability: 8
GPU memory: 23034
Using low-memory setup (40 GB GPU RAM)
Adjusting pair_transition_shard_spec in model_config.py
Enabling unified memory
Executing run_alphafold.py
I0919 08:08:00.961979 138879790076544 xla_bridge.py:895] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I0919 08:08:00.963072 138879790076544 xla_bridge.py:895] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
Running AlphaFold 3. Please note that standard AlphaFold 3 model parameters are
only available under terms of use provided at
https://github.com/google-deepmind/alphafold3/blob/main/WEIGHTS_TERMS_OF_USE.md.
If you do not agree to these terms and are using AlphaFold 3 derived model
parameters, cancel execution of AlphaFold 3 inference with CTRL-C, and do not
use the model parameters